In [1]:
import pandas as pd
import os
from datasets import Dataset as HFDataset, Audio

In [2]:
def prep_df(root_path):
    # Read the TSV file and extract data
    tsv_file_path = os.path.join(root_path, "utt_spk_text.tsv")
    with open(tsv_file_path, "r", encoding="utf-8") as tsv_file:
        lines = tsv_file.readlines()

    # Prepare data for the Hugging Face dataset
    file_paths = []
    folder_names = []
    texts = []

    for line in lines:
        file_name, _, text = line.strip().split("\t")
        folder_name = file_name[:2]

        file_path = os.path.join(root_path, "data", folder_name, file_name + ".flac")

        file_paths.append(file_path)
        folder_names.append(folder_name)
        texts.append(text)

    # Create the Hugging Face dataset
    dataset_dict = {
        "file_path": file_paths,
        "folder_name": folder_names,
        "text": texts,
    }
    
    # create dataframe
    df = pd.DataFrame(dataset_dict)
    
    return df


In [3]:
df = prep_df("/home/venom/repo/Bengali.AI-Speech-Recognition/openslr/")

In [4]:
print(df.head())

                           file_path folder_name                     text
0  ./openslr/data/00/000020a912.flac          00  বাংলাদেশে দায়িত্ব নেবে
1  ./openslr/data/00/000039928e.flac          00      এ ধরণের কার্ড নিয়ে
2  ./openslr/data/00/00005debc7.flac          00        হতে উপার্জিত অর্থ
3  ./openslr/data/00/00009e687c.flac          00    হাসির বিষয় হয়েই আছে
4  ./openslr/data/00/00012843bc.flac          00          সার্ক দেশগুলোতে


In [5]:
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor


# Load the tokenizer and feature extractor
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="bengali", task="transcribe")
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="bengali", task="transcribe")

In [6]:
import torch
import librosa
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split


fraction = 0.2

num_samples = len(df)
num_val_samples = min(128, int(fraction * num_samples))
num_test_samples = int(fraction * num_samples)
num_train_samples = num_samples - num_val_samples - num_test_samples

# Split the dataset into train, test, and validation sets.
train_set, remaining_set = train_test_split(df, train_size=num_train_samples, random_state=42)
test_set, val_set = train_test_split(remaining_set, train_size=num_test_samples, random_state=42)

sampling_rate = 16000
path_template = "/home/venom/repo/Bengali.AI-Speech-Recognition/{}"

In [7]:
from datasets import IterableDataset

def dataset_generator(df):
    for _, row in df.iterrows():
        audio_array = librosa.load(path_template.format(row["file_path"]))[0]
        yield {
            "input_features": feature_extractor(audio_array, sampling_rate=sampling_rate).input_features[0], 
            "labels": tokenizer(row["text"]).input_ids
        }

In [8]:
train_ds = IterableDataset.from_generator(dataset_generator, gen_kwargs={"df": train_set})
test_ds = IterableDataset.from_generator(dataset_generator, gen_kwargs={"df": test_set})
eval_ds = IterableDataset.from_generator(dataset_generator, gen_kwargs={"df": val_set})

In [9]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [10]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [11]:
import evaluate

metric = evaluate.load("wer")


2023-07-24 19:49:45.332245: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-24 19:49:45.569143: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [13]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")


In [14]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []


In [15]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-test",  # change to a repo name of your choice
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=1,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)


In [16]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)


In [17]:

trainer.train()

/home/venom/miniconda3/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/4000 [00:00<?, ?it/s]

`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


OutOfMemoryError: CUDA out of memory. Tried to allocate 104.00 MiB (GPU 0; 5.80 GiB total capacity; 3.88 GiB already allocated; 49.19 MiB free; 4.40 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF